In [ ]:
import httpx

# Define the request payload
payload = {
    "model": "gpt-4o-mini",
    "messages": [
        {"role": "system", "content": "Be helpulf"},
        {"role": "user", "content": "Hello, how are you?"}
    ],
    "temperature": 0.7,
    "stream": True  # Include this if your backend requires it explicitly
}

# Query parameters
params = {
    "user_id": "user-123",
    "session_id": "session-456",
    "channel_id": "channel-789",
    "channel_type": "web",
    "api_provider": "openai",
    "device_info": "ZGV2aWNlPXdlYjtzb3VyY2U9YnJvd3Nlcg=="
}

# Bearer token
bearer_token = "postgres"
host = 'http://127.0.0.1:5008'


# Stream the response
with httpx.stream(
    "POST",
    f"{host}/chat/completions",
    json=payload,
    params=params,
    headers={"Authorization": f"Bearer {bearer_token}"},
    timeout=None  # Disable timeout for long streaming responses
) as response:
    print("Status code:", response.status_code)
    print("Streaming response:")
    for line in response.iter_lines():
        if line:
            print(line)
            #print(line.decode("utf-8"))


In [ ]:
#! touch /Users/sirsh/Downloads/example.txt && echo "hello world" >> /Users/sirsh/Downloads/example.txt


In [2]:
import httpx

# File to upload
file_path = "/Users/sirsh/Downloads/example.txt"
task_id = "task-abc-123"
bearer_token = "postgres"

# Open the file and prepare the multipart/form-data
with open(file_path, "rb") as f:
    files = {
        "file": ("example.txt", open(file_path, "rb"), "text/plain")
    }
    data = {
        "task_id": task_id
    }
    
    response = httpx.post(
        "http://127.0.0.1:5000/admin/content/upload",
        files=files,
        data=data,
        headers={
            "Authorization": f"Bearer {bearer_token}"
        },
        timeout=30.0
    )

# Print the response
print("Status code:", response.status_code)
print("Response body:", response.text)


In [ ]:
from percolate.services.S3Service import S3Service
s3_service = S3Service()

result = s3_service.upload_file(
            project_name='test',
            file_name='test.txt',
            file_content=tmp_file,
            content_type=file.content_type
        )

In [8]:
from pathlib import Path
from percolate.services.S3Service import S3Service
import hashlib

file_path = Path("/Users/sirsh/Downloads/example.txt")
s3_service = S3Service()

with open(file_path,'rb') as f:
    
    file_bytes = f.read()
    print("SHA256:", hashlib.sha256(file_bytes).hexdigest())
    print("Size:", len(file_bytes))
    f.seek(0)
    
    result = s3_service.upload_file(
        project_name="test",
        file_name=file_path.name,
        file_content=f,  # send bytes
        content_type="text/plain"
    )

print("Upload result:", result)

In [ ]:
# Audio mode

In [24]:
def record_audio(
    filename: str = "audio.wav",
    duration: float = 5.0,
    fs: int = 16000
) -> str:
    """
    Records audio from the default microphone and saves it to a WAV file.
    pip install sounddevice scipy requests
    Parameters:
        filename (str): Output WAV file name.
        duration (float): Recording duration in seconds.
        fs (int): Sampling frequency.

    Returns:
        str: The path to the saved WAV file.
    """
    import sounddevice as sd
    from scipy.io.wavfile import write
    import base64
    def encode_audio_base64(
        filename: str  ) -> str:
        """
        Reads a WAV file and returns its Base64-encoded string.

        Parameters:
            filename (str): Path to the WAV file.

        Returns:
            str: Base64-encoded audio content.
        """
        with open(filename, "rb") as f:
            encoded = base64.b64encode(f.read()).decode('utf-8')
        return encoded

    print(f"Recording {duration} seconds of audio to '{filename}'...")
    recording = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')
    sd.wait()  # Wait until recording is finished
    write(filename, fs, recording)
    print(f"Saved recording to '{filename}'.")
    return encode_audio_base64(filename)



def test_transcription(
    duration: float = 5.0,
    fs: int = 16000,
    model: str = "whisper-1", 
    temperature: float = 0.0,
    response_format: str = "json",
    language: str = None
) -> dict:
    """
    End-to-end test: record audio, encode to Base64, and transcribe using Whisper.

    Parameters:
        duration (float): Recording duration in seconds.
        fs (int): Sampling rate.
        model (str): Whisper model name.
        api_key (str): OpenAI API key override.
        temperature (float): Sampling temperature.
        response_format (str): Response format.
        language (str): Language hint.

    Returns:
        dict: Transcription result.
    """
    from percolate.services.llm.utils import audio_to_text
 
    """get some test data"""
    b64_audio = record_audio(duration=duration, fs=fs)
 
    result = audio_to_text(
        base64_audio=b64_audio,
        model=model, 
        temperature=temperature,
        response_format=response_format,
        language=language
    )
    # Print result
    text = result.get("text") if isinstance(result, dict) else None
    print("Transcription:", text or result)
    return result

In [17]:
b64_audio = record_audio(duration=2, fs=8000)
len(b64_audio)

In [26]:
b64_audio[:10],b64_audio[-10:] 

In [23]:
from percolate.services.llm.utils import audio_to_text
audio_to_text(b64_audio)

In [30]:
import httpx

# Define the request payload
payload = {
    "model": "gpt-4o-mini",
    "messages": [
        {"role": "system", "content": "Be helpulf"},
        {"role": "user", "content": b64_audio}
    ],
    "temperature": 0.7,
    "stream": True  # Include this if your backend requires it explicitly
}

# Query parameters
params = {
    "user_id": "user-123",
    "session_id": "session-456",
    "channel_id": "channel-789",
    "channel_type": "web",
    "api_provider": "openai",
    #here we say its audio
    "is_audio": True,
    "device_info": "ZGV2aWNlPXdlYjtzb3VyY2U9YnJvd3Nlcg=="
}

# Bearer token
bearer_token = "postgres"
host = 'http://127.0.0.1:5000'


# Stream the response
with httpx.stream(
    "POST",
    f"{host}/chat/completions",
    json=payload,
    params=params,
    headers={"Authorization": f"Bearer {bearer_token}"},
    timeout=None  # Disable timeout for long streaming responses
) as response:
    print("Status code:", response.status_code)
    print("Streaming response:")
    for line in response.iter_lines():
        if line:
            print(line)
            #print(line.decode("utf-8"))
